In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import networkx as nx

from node2vec import Node2Vec

from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from scipy.optimize import  linear_sum_assignment
import numpy as np
import matplotlib.pyplot as plt

/home/jw2529/.conda/envs/py39/lib/python3.9/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
from seirsplus.assignment import embed_nodes, get_equal_sized_clusters

In [4]:
def get_household(households, node_id):
    for household in households:
        if node_id in household['indices']:
            return household['indices']
    return None


def get_assignment(groups, node_id):
    for group in groups.values():
        if node_id in group:
            return group


In [5]:
from seirsplus.networks import generate_demographic_contact_network, household_country_data

N = 1000
np.random.seed(0)
demographic_graphs, individual_ageGroups, households = generate_demographic_contact_network(
    N=N, 
    demographic_data=household_country_data('US'),
    distancing_scales=[0.7], 
    isolation_groups=[]
)

G = demographic_graphs['baseline']

Generated overall age distribution:
0-9: 0.1140	(-0.0070 from target)
10-19: 0.1310	(0.0000 from target)
20-29: 0.1410	(0.0040 from target)
30-39: 0.1440	(0.0110 from target)
40-49: 0.1170	(-0.0070 from target)
50-59: 0.1290	(-0.0020 from target)
60-69: 0.1020	(-0.0130 from target)
70-79: 0.0720	(0.0020 from target)
80+: 0.0500	(0.0120 from target)

Generated household size distribution:
1: 0.3259	(0.0422 from target)
2: 0.2988	(-0.0463 from target)
3: 0.1160	(-0.0346 from target)
4: 0.1481	(0.0205 from target)
5: 0.0691	(0.0114 from target)
6: 0.0346	(0.0119 from target)
7: 0.0074	(-0.0051 from target)
Num households: 405
mean household size: 2.448393594

Generating graph for 0-9...


/home/jw2529/.conda/envs/py39/lib/python3.9/site-packages/networkx/linalg/graphmatrix.py:187: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return adjacency_matrix(G, nodelist, dtype, weight)


Generating graph for 10-19...
Generating graph for 20-59...
Generating graph for 60+...


In [6]:
G.edges.data()

EdgeDataView([(0, 1, {'weight': 1}), (0, 2, {'weight': 1}), (0, 3, {'weight': 1}), (0, 4, {'weight': 1}), (0, 5, {'weight': 1}), (0, 8, {'weight': 1}), (0, 55, {'weight': 1}), (0, 246, {}), (1, 2, {'weight': 1}), (1, 3, {'weight': 1}), (1, 4, {'weight': 1}), (1, 5, {'weight': 1}), (1, 8, {'weight': 1}), (1, 11, {'weight': 1}), (1, 13, {'weight': 1}), (1, 15, {'weight': 1}), (1, 17, {'weight': 1}), (1, 29, {'weight': 1}), (1, 37, {'weight': 1}), (1, 38, {'weight': 1}), (1, 40, {'weight': 1}), (1, 44, {'weight': 1}), (1, 56, {'weight': 1}), (1, 70, {'weight': 1}), (1, 249, {}), (2, 3, {'weight': 1}), (2, 4, {'weight': 1}), (2, 5, {'weight': 1}), (2, 6, {'weight': 1}), (2, 7, {'weight': 1}), (2, 8, {'weight': 1}), (2, 9, {'weight': 1}), (2, 11, {'weight': 1}), (2, 14, {'weight': 1}), (2, 16, {'weight': 1}), (2, 17, {'weight': 1}), (2, 18, {'weight': 1}), (2, 28, {'weight': 1}), (2, 29, {'weight': 1}), (2, 37, {'weight': 1}), (2, 41, {'weight': 1}), (2, 44, {'weight': 1}), (2, 71, {'weight

In [7]:
embedding, node2vec_model = embed_nodes(G)
clusters = get_equal_sized_clusters(
    X=embedding,
    model=node2vec_model,
    graph=G,
    cluster_size=5,
)  # dict of node_id to cluster id
cluster_ids = list(set(clusters.values()))  # unique list of cluster ids
groups = {
    i: [x for x, v in clusters.items() if v == i] for i in cluster_ids
}  # dict of cluster ids as the keys and the node ids as the values

Computing transition probabilities:   0%|          | 0/1000 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:03<00:00,  2.96it/s]
/home/jw2529/.conda/envs/py39/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [8]:
for i in range(100):
    print(get_household(households, i), get_assignment(groups, i))

[0, 246] [0, 1, 11, 37, 44]
[1, 249] [0, 1, 11, 37, 44]
[2, 252, 251] [2, 41, 70, 71, 200]
[3, 253] [3, 13, 35, 54, 55]
[5, 4] [4, 10, 14, 22, 62]
[5, 4] [5, 6, 7, 8, 15]
[6, 115] [5, 6, 7, 8, 15]
[116, 8, 7, 263] [5, 6, 7, 8, 15]
[116, 8, 7, 263] [5, 6, 7, 8, 15]
[9, 117, 266, 265, 264] [9, 18, 56, 67, 521]
[11, 10, 120] [4, 10, 14, 22, 62]
[11, 10, 120] [0, 1, 11, 37, 44]
[12, 796, 281, 280, 279, 795] [12, 48, 796, 850, 883]
[13, 291, 290, 289] [3, 13, 35, 54, 55]
[15, 14] [4, 10, 14, 22, 62]
[15, 14] [5, 6, 7, 8, 15]
[17, 16] [16, 23, 81, 99, 100]
[17, 16] [17, 85, 102, 393, 736]
[123, 18, 322, 321, 320] [9, 18, 56, 67, 521]
[19, 325, 324, 323] [19, 47, 63, 77, 571]
[124, 20] [20, 25, 49, 50, 178]
[21, 125, 326] [21, 68, 88, 149, 198]
[129, 22, 128] [4, 10, 14, 22, 62]
[133, 23] [16, 23, 81, 99, 100]
[24, 138] [24, 39, 40, 454, 455]
[25, 140, 139, 362] [20, 25, 49, 50, 178]
[26, 141] [26, 66, 442, 579, 580]
[28, 27, 368, 367, 366] [27, 28, 366, 367, 368]
[28, 27, 368, 367, 366] [27,

In [9]:
for e in G.edges():
    if 'weight' not in G[e[0]][e[1]]:
        G[e[0]][e[1]]['weight'] = 10000000000
        

In [10]:
G.edges.data()

EdgeDataView([(0, 1, {'weight': 1}), (0, 2, {'weight': 1}), (0, 3, {'weight': 1}), (0, 4, {'weight': 1}), (0, 5, {'weight': 1}), (0, 8, {'weight': 1}), (0, 55, {'weight': 1}), (0, 246, {'weight': 10000000000}), (1, 2, {'weight': 1}), (1, 3, {'weight': 1}), (1, 4, {'weight': 1}), (1, 5, {'weight': 1}), (1, 8, {'weight': 1}), (1, 11, {'weight': 1}), (1, 13, {'weight': 1}), (1, 15, {'weight': 1}), (1, 17, {'weight': 1}), (1, 29, {'weight': 1}), (1, 37, {'weight': 1}), (1, 38, {'weight': 1}), (1, 40, {'weight': 1}), (1, 44, {'weight': 1}), (1, 56, {'weight': 1}), (1, 70, {'weight': 1}), (1, 249, {'weight': 10000000000}), (2, 3, {'weight': 1}), (2, 4, {'weight': 1}), (2, 5, {'weight': 1}), (2, 6, {'weight': 1}), (2, 7, {'weight': 1}), (2, 8, {'weight': 1}), (2, 9, {'weight': 1}), (2, 11, {'weight': 1}), (2, 14, {'weight': 1}), (2, 16, {'weight': 1}), (2, 17, {'weight': 1}), (2, 18, {'weight': 1}), (2, 28, {'weight': 1}), (2, 29, {'weight': 1}), (2, 37, {'weight': 1}), (2, 41, {'weight': 1})

In [11]:
embedding, node2vec_model = embed_nodes(G)
clusters = get_equal_sized_clusters(
    X=embedding,
    model=node2vec_model,
    graph=G,
    cluster_size=5,
)  # dict of node_id to cluster id
cluster_ids = list(set(clusters.values()))  # unique list of cluster ids
groups = {
    i: [x for x, v in clusters.items() if v == i] for i in cluster_ids
}  # dict of cluster ids as the keys and the node ids as the values

Computing transition probabilities:   0%|          | 0/1000 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:02<00:00,  3.63it/s]
/home/jw2529/.conda/envs/py39/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:
for i in range(100):
    print(get_household(households, i), get_assignment(groups, i))

[0, 246] [0, 105, 106, 959, 986]
[1, 249] [1, 249, 456, 505, 586]
[2, 252, 251] [2, 251, 252, 285, 575]
[3, 253] [3, 664, 672, 834, 957]
[5, 4] [4, 24, 36, 138, 913]
[5, 4] [5, 62, 196, 559, 560]
[6, 115] [6, 115, 189, 190, 436]
[116, 8, 7, 263] [7, 8, 116, 247, 263]
[116, 8, 7, 263] [7, 8, 116, 247, 263]
[9, 117, 266, 265, 264] [9, 117, 264, 265, 266]
[11, 10, 120] [10, 11, 120, 241, 958]
[11, 10, 120] [10, 11, 120, 241, 958]
[12, 796, 281, 280, 279, 795] [12, 279, 281, 795, 796]
[13, 291, 290, 289] [13, 289, 290, 291, 498]
[15, 14] [14, 182, 612, 624, 898]
[15, 14] [15, 629, 677, 960, 961]
[17, 16] [16, 485, 486, 895, 896]
[17, 16] [17, 52, 863, 864, 903]
[123, 18, 322, 321, 320] [18, 123, 320, 321, 322]
[19, 325, 324, 323] [19, 323, 324, 325, 329]
[124, 20] [20, 124, 605, 911, 955]
[21, 125, 326] [21, 125, 326, 924, 990]
[129, 22, 128] [22, 30, 128, 129, 939]
[133, 23] [23, 94, 133, 755, 995]
[24, 138] [4, 24, 36, 138, 913]
[25, 140, 139, 362] [25, 139, 140, 362, 578]
[26, 141] [26,